## **Face_Mask_Detection_App_Model**

In [ ]:
pip install tensorflow==2.1.0

In [ ]:
pip install keras==2.3.1

In [ ]:
#Training Model

In [ ]:
# Import Labraries
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint

In [ ]:
# Import Visualization Libraries
import matplotlib.pyplot as plt 

In [ ]:
# Import tensorflow
import tensorflow

In [ ]:
print(tensorflow.__version__)

In [ ]:
print(keras.__version__)

In [ ]:
# path train  and Validation data
train_data_path = "/content/drive/My Drive/Colab Notebooks/Face_Mask_Detection_App/dataset/train"
Validation_data_path = "/content/drive/My Drive/Colab Notebooks/Face_Mask_Detection_App/dataset/valid"

In [ ]:
# show augmented images function
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20, 20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

In [ ]:
# augmentation configuration we will use for training
# It generate more images using below parameters
training_datagen = ImageDataGenerator(rescale=1./255,
                                      rotation_range=40,
                                      width_shift_range=0.2,
                                      height_shift_range=0.2,
                                      shear_range=0.2,
                                      zoom_range=0.2,
                                      horizontal_flip=True,
                                      fill_mode='nearest')

# this is a geneartor that will read pictures found in at train_data_path,
# and indefinitely generate batches of augmented image data
training_data = training_datagen.flow_from_directory(train_data_path, target_size=(200,200),
                                                     batch_size = 128,
                                                     class_mode = 'binary')

In [ ]:
training_data.class_indices

In [ ]:
# augmentation configuration we will use for validation --> only rescaling
validation_datagen = ImageDataGenerator(rescale=1./255)

# this is a geneartor that will read pictures found in at validation_data_path,
# and indefinitely generate batches of augmented image data
validation_data = validation_datagen.flow_from_directory(Validation_data_path,
                                  target_size=(200,200),
                                  batch_size=128,
                                  class_mode='binary')

In [ ]:
# Show Augmented Images
images = [training_data[0][0][0] for i in range(5)]
plotImages(images)

In [ ]:
# Save best Model using Vall accuracy
save_model_path = '/content/drive/My Drive/Colab Notebooks/Face_Mask_Detection_App/model/model.h5'
checkpoint = ModelCheckpoint(save_model_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

## **Building CNN Model**

In [ ]:
cnn_model = keras.models.Sequential([
                                      keras.layers.Conv2D(filters=32, kernel_size=5, input_shape=[200, 200, 3]),
                                      keras.layers.MaxPooling2D(pool_size=(4,4)),
                                      keras.layers.Conv2D(filters=64, kernel_size=4),
                                      keras.layers.MaxPooling2D(pool_size=(3,3)),
                                      keras.layers.Conv2D(filters=128, kernel_size=3),
                                      keras.layers.MaxPooling2D(pool_size=(2,2)),                                    
                                      keras.layers.Conv2D(filters=256, kernel_size=2),
                                      keras.layers.MaxPooling2D(pool_size=(2,2)),
                                      keras.layers.Dropout(0.5),
                                      keras.layers.Flatten(), # neural network building
                                      keras.layers.Dense(units=128, activation='relu'), # input layer
                                      keras.layers.Dropout(0.1),
                                      keras.layers.Dense(units=256, activation= 'relu'),
                                      keras.layers.Dropout(0.25),
                                      keras.layers.Dense(units=2, activation='softmax') # output layer

])

## **Compile CNN Model**

In [ ]:
#cnn_model.compile(optimizer = Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
cnn_model.compile(optimizer= Adam(lr = 0.001), loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

## **Train CNN Model**

In [ ]:
history = cnn_model.fit(training_data, epochs=50, verbose=1, validation_data=validation_data, callbacks=callbacks_list)

## **Plots Loss & Accuracy** 

In [ ]:
import matplotlib.pyplot as plt
# plot loss
plt.plot(cnn_model.history.history['loss'], label='train_loss')
plt.plot(cnn_model.history.history['val_loss'], label='val_loss')
plt.legend()
plt.show()
plt.savefig('Loss_Val')
 
# plot accuracy
plt.plot(cnn_model.history.history['accuracy'], label='train_acc')
plt.plot(cnn_model.history.history['val_accuracy'], label='val_acc')
plt.legend()
plt.show()
plt.savefig('Accuracy_Val')